In [ ]:
# Install py3Dmol using pip
!pip install py3Dmol
# Install biopython using pip
!pip install biopython
# Install pdb2pqr v3.0 using pip
!pip install pdb2pqr
# We will also install kora for using RDkit
!pip install kora
# Install vina usiung pip
!pip install vina

In [ ]:
#Install conda using the new conda-colab library
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

#Install MGLtools and OpenBabel from
#the bioconda repository
!conda install -c conda-forge -c bioconda mgltools openbabel zlib --yes

⏬ Downloading https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:22
🔁 Restarting kernel...
Solving environment: - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mgltools
    - openbabel
    - zlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.9.24  |       ha878542_0         150 KB  conda-forge
    cairo-1.16.0               |       h19f5f5c_2         1.1 MB
    certifi-2022.9.24          |     pyhd8ed1ab_0         155 KB  conda-forge
    conda-22.9.0               |   py37h89c1867_1         960 KB  co

In [ ]:
#Downloading the PDB files using biopython
import os
from Bio.PDB import *
pdbid = ['1t5c']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
#This script will create a folder called "single-docking" for our experiment
#Then, it will print all "ATOM" and "TER" lines from a given PDB into a new file

#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path 

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
crosspath = Path("/content/cross-dock/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(crosspath):
  print("path already exists")
if not os.path.exists(crosspath):
  os.mkdir(crosspath)
  print("path was succesfully created")

#Now we assign a variable "protein" with the name and extension of our pdb
protein = "1t5c.pdb"

#And we use the following script to selectively print the lines that contain the
#string "ATOM" and "TER" into a new file inside our recently created folder
with open(crosspath / "1t5c_prot.pdb","w") as g:
  f = open(protein,'r')
  for line in f:
    row = line.split()
    if row[0] == "ATOM": # and row[4] == "A":
      g.write(line)
    elif row[0] == "TER":
      g.write("TER\n")
  g.write("END")
  print("file successfully created")

path already exists
file successfully created


In [ ]:
#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#We will first create a path for all ligands that we will use in this tutorial
#Notice that the HOME folder for a hosted runtime in colab is /content/
ligandpath = Path("/content/ligands/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(ligandpath):
  print("ligand path already exists")
if not os.path.exists(ligandpath):
  os.mkdir(ligandpath)
  print("ligand path was succesfully created")

ligand path already exists


In [ ]:
#Downloading NExT 3500 diversity library from uploaded file in sdf format
from google.colab import drive
drive.mount('/content/drive')

# Test code
# !cp 'drive/MyDrive/test.sdf' $ligandpath

!cp 'drive/MyDrive/Diversity_3500_Library.sdf' $ligandpath

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)


In [ ]:
#Use the following viewer to load your SMILES as a 3D molecule
import py3Dmol
import kora.install.rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view()
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

from ipywidgets import interact,fixed,IntSlider
import ipywidgets

def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None

@interact
def smi2viewer(smi='CC=O'):
    try:
        conf = smi2conf(smi)
        return MolTo3DView(conf).show()
    except:
        return None

interactive(children=(Text(value='CC=O', description='smi'), Output()), _dom_classes=('widget-interact',))

In [ ]:
# Prepare the protein and ligands
import subprocess

# subprocess.run(['prepare_receptor4.py', '-r', '/content/cross-dock/1t5c_prot.pdb', '-o', '/content/cross-dock/1t5c_prot.pdbqt', '-A', 'hydrogens', '-U', 'nphs_lps', '-v'])
# subprocess.call(f'prepare_receptor4.py -r /content/cross-dock/1t5c_prot.pdb -o /content/cross-dock/1t5c_prot.pdbqt -A hydrogens -U nphs_lps -v', shell = True)
subprocess.run(['pdb2pqr30', '--ff', 'AMBER', '--keep-chain', '--titration-state-method', 'propka', '--with-ph', '7.4', '/content/cross-dock/1t5c_prot.pdb', '/content/cross-dock/1t5c_prot.pqr'])
subprocess.run(['prepare_receptor4.py', '-r', '/content/cross-dock/1t5c_prot.pqr', '-o', '/content/cross-dock/1t5c_prot.pdbqt', '-C', '-U', 'nphs_lps', '-v'])

CompletedProcess(args=['prepare_receptor4.py', '-r', '/content/cross-dock/1t5c_prot.pqr', '-o', '/content/cross-dock/1t5c_prot.pdbqt', '-C', '-U', 'nphs_lps', '-v'], returncode=0)

In [ ]:
import kora.install.rdkit
from rdkit import Chem

# Test code
# sdf_path = "/content/ligands/test.sdf"

# Read sdf by rdkit
sdf_path = "/content/ligands/Diversity_3500_Library.sdf"
suppl = Chem.SDMolSupplier(sdf_path, removeHs = False)
# ligs = [mol for mol in suppl if mol is not None]

In [ ]:
from vina import Vina
from rdkit.Chem.rdMolTransforms import ComputeCentroid

# Prepare ligands within the crossing docking
s = open("/content/scores.txt", "w")
id = 1

for mol in suppl:
  if mol is not None:
    centroid = ComputeCentroid(mol.GetConformer())
    # print(centroid.x, centroid.y, centroid.z)
    mol_id = mol.GetProp('NSC#')
    print(str(mol_id))
    s.write(str(mol_id))
    s.write(' ')
    smi = mol.GetProp('SMILES')
    print(smi)
    s.write(smi)
    s.write(' ')

    # Write smiles file
    with open(ligandpath / 'ligand.smi', 'w') as f:
      f.write(smi)
    subprocess.run(['obabel', '/content/ligands/ligand.smi', '-O', 'ligand.mol2', '--gen3d', 'best', '-p', '7.4', '--canonical'])
    # os.remove('/content/ligands/ligand.smi')
    subprocess.run(['prepare_ligand4.py', '-l', 'ligand.mol2', '-o', '/content/ligands/ligand.pdbqt', '-U', 'nphs_lps', '-v'])
    # os.remove('ligand.mol2')

    # Setup AutoDock Vina
    v = Vina(sf_name = 'vina', cpu = 8)
    v.set_receptor('/content/cross-dock/1t5c_prot.pdbqt')
    v.set_ligand_from_file('/content/ligands/ligand.pdbqt')
    v.compute_vina_maps(center = [centroid.x, centroid.y, centroid.z], box_size = [30, 30, 30])

    # Score the current pose
    energy = v.score()
    # print('Score before minimization: %.3f (kcal/mol)' % energy[0])

    # Minimized locally the current pose
    energy_minimized = v.optimize()
    # print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
    # v.write_pose(f'1t5c_ligand{id}_minimized.pdbqt', overwrite = True)

    # Dock the ligand
    v.dock(exhaustiveness = 8, n_poses = 9)
    dock_energy = v.score()
    print('Score after docking : %.3f (kcal/mol)' % dock_energy[0])
    s.write('%.3f\n' % dock_energy[0])
    # v.write_poses(f'1t5c_ligand{id}_vina_out.pdbqt', n_poses = 9, overwrite = True)

    # Remove ligand pdbqt file
    # os.remove('/content/ligands/ligand.pdbqt')

    # Update ligand id
    id += 1

s.close()

2579982
COC(=O)CCC(C)C1CCC2C3CCC4C[C@@H](CCC4(C)C3CC(=O)C12C)OC(=O)c1ccccc1
Score after docking : -3.521 (kcal/mol)
2505256
CC(=O)O[C@H]1[C@H](C[C@H]2[C@@H]3CC[C@H]4C[C@H](OC(C)=O)[C@H](C[C@]4(C)[C@H]3CC[C@]12C)N1CCCCC1)[N+]1(C)CCCCC1
Score after docking : -3.518 (kcal/mol)
2579969
C[C@]12CCC3C(CCC4CCCC[C@]34C)C1CCC2N
Score after docking : -3.541 (kcal/mol)
2552946
COC(=O)c1sccc1NC(=O)CCC(O)=O
Score after docking : -3.728 (kcal/mol)
2523997
COC(=O)c1sccc1NC(=O)CC1(CCCCCC1)C(O)=O
Score after docking : -2.878 (kcal/mol)
2527051
Cc1ccccc1NC(=O)CC1(CCCCC1)C(O)=O
Score after docking : -2.865 (kcal/mol)
2560429
CC(=C)CC1(CCCCC1)Nc1ccccc1C
Score after docking : -4.180 (kcal/mol)
2559454
CC(C)(C)c1ccccc1NC(=O)CCC(O)=O
Score after docking : -3.791 (kcal/mol)
2534103
Cc1ccccc1NC(=O)COC(=O)COc1ccccc1
Score after docking : -1.447 (kcal/mol)
2561037
CC(=O)NN=C(S)Nc1ccccc1C(F)(F)F
Score after docking : -5.204 (kcal/mol)
2559587
OC(=O)C1C2CC(C=C2)C1C(=O)Nc1ccccc1C(F)(F)F
Score after docking : -3.334 